# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [83]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [84]:
# TODO: Import the necessary libs
# For example: 
import os
lib_dir = os.path.join(os.getcwd(), 'lib')
if lib_dir not in sys.path:
    sys.path.append(lib_dir)
from lib.agents import Agent
from dotenv import load_dotenv
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from typing import TypedDict, List, Optional, Union
import json
from dotenv import load_dotenv
from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Run
from lib.llm import LLM
from lib.messages import AIMessage, UserMessage, SystemMessage, ToolMessage
from lib.tooling import Tool, ToolCall, tool
import chromadb

In [85]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [86]:
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection(
    name="udaplay11"
)


### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [87]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay2")
from lib.tooling import tool
@tool
def retrieve_game(query: str) -> str:
    """
    Semantic search: Finds most results in the vector DB.

    args:
    - query: a question about the game industry.

    You'll receive results as a list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    # Perform a semantic search in the vector database
    return "\n".join(collection.query(
       query_texts=[query],
       n_results=10,
       include=['documents']
       )["documents"][0])

print(retrieve_game("game boy"))

[Game Boy Advance] Pokémon Ruby and Sapphire (2002) - Third-generation Pokémon games set in the Hoenn region, featuring new Pokémon and double battles.
[Game Boy Color] Pokémon Gold and Silver (1999) - Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.
[GameCube] Super Smash Bros. Melee (2001) - A crossover fighting game featuring characters from various Nintendo franchises battling it out in dynamic arenas.
[Super Nintendo Entertainment System (SNES)] Super Mario World (1990) - A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser.
[PlayStation 2] Grand Theft Auto: San Andreas (2004) - An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson.
[Nintendo 64] Super Mario 64 (1996) - A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach.
[Xbox One] Minecraft (2014) - A sandbox g

#### Evaluate Retrieval Tool

In [88]:
from lib.tooling import tool

@tool
def format_evaluation(useful: bool, description: str):
    return {
        'useful': useful,
        'description': description
    }


# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result
@tool
def evaluate_retrieval(query: str, documents: str):
    chat_model = LLM(api_key=os.getenv('OPENAI_API_KEY'), tools=[format_evaluation])
    docList = documents.split('\n')
    messages = [
        SystemMessage(content="""
        You're an AI agent tasked with evaluating the usefulness of retrieved documents.
        Your task is to evaluate if the documents are enough to respond the query.
        Give a detailed explanation, so it's possible to take an action to accept it or not.
        Format your response by calling the format_evaluation tool.
        """),
        UserMessage(content="Query: " + query + ". " + "Documents: " + '\n'.join(docList))
    ]
    response = chat_model.invoke(messages)
    return response.tool_calls[0].function.arguments

print(evaluate_retrieval("game boy", retrieve_game("game boy")))

{"useful":false,"description":"The retrieved documents primarily focus on various games across different platforms, but they do not provide sufficient information specifically about the Game Boy itself. While there are mentions of games associated with the Game Boy, such as Pokémon Gold and Silver for the Game Boy Color and Pokémon Ruby and Sapphire for the Game Boy Advance, there is no comprehensive information about the Game Boy's features, history, or significance. Therefore, the documents do not adequately respond to the query regarding the Game Boy."}


#### Game Web Search Tool

In [89]:
from tavily import TavilyClient
from lib.tooling import tool

# Tool Docstring:
#    Search the internet for information about video games.
#    args:
#    - query: a question about game industry. 
@tool
def game_web_search(query: str) -> list[str]:
    tavily_client = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))
    response = tavily_client.search(query)
    return [result['content'] for result in response.get('results', [])]

print(game_web_search("best game boy game"))

['**Like many gamers my age, the Game Boy was the first handheld system I owned, and I still remember it fondly.** I got it for Christmas in 4th grade with both Link’s Awakening (as a pack-in) and Wario Land. I saw it on a few other “best Game Boy games” lists and decided to order a cart and manual from eBay. While nowhere near as refined as beat ’em up entries like the exquisite Turtles in Time, it’s a solid action title! I love Six Golden Coins-it’s not just one of the best Game Boy games but easily one of my top 10 Mario games! The Switch remake was also a joy to play, but I will always prefer the original release on Game Boy for the nostalgia.', "As an 8 year old kid in 1989, the first time I ever saw a Game Boy my cousin was using what is essentially a book light to play Tetris on this blurry, pea-soup-green screen. Probably the best game I have played on the GB so far. The first 3 SaGa games (Final Fantasy Legend 1-3) are some of my favourite original GB games. Since you have an 

### Agent

In [90]:
tools = [game_web_search, retrieve_game, evaluate_retrieval]

agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
        "You're an AI Agent tasked with mining insights about video games."
        "You have access to multiple tools to help you answer user questions. "
        "Always recall your background knowledge about games."
        "Give very careful answers and do research."
        "Use the results of your research only if you evaluate them as useful."
        "You follow a pattern of of Thought and Action. "
        "Create a plan of execution: "
        "- Use Thought to describe your thoughts about the question you have been asked. "
        "- Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly."
        "When you think it's over, return the answer "
        "Never try to respond directly if the question needs a tool. "
        "But if you don't have a tool available, you can respond directly. "
        f"The actions you have are the Tools: {tools}. \n"
    ),
    tools=tools
)

In [81]:
run_object = agent.invoke(
    query="When were Pokémon Gold and Silver released, and what pokemon is on the label of each?",
)

msgs = run_object.get_final_state()["messages"]
for m in msgs:
    print(f"{m.__class__.__name__}: {m.content}\n")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
SystemMessage: You're an AI Agent tasked with mining insights about video games.You have access to multiple tools to help you answer user questions. Always recall your background knowledge about games.Give very careful answers and do research.Use the results of your research only if you evaluate them as useful.You follow a pattern of of Thought and Action. Create a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. if you don't have a tool available, you can respond directly.When you think it's over, return the answer Never try to respond 

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes